In [1]:
import pandas as pd
import os
import json
import numpy as np
# import streamlit as st
# import streamlit.components.v1 as components
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import sys
DIR = os.getenv('KEA_BASE_DIR')
sys.path.append(DIR)
import application
from application import *

In [60]:
# Reading the GeoJSON
with open("AC_Boundary_simplified.json",'r',encoding='utf-8') as file:
    acmap = json.loads(file.read())

In [61]:
# This part of the JSON contains critical information
acmap['features'][0]['properties']

{'OBJECTID': 610,
 'AC_CODE': '1',
 'AC_NAME': 'Nippani',
 'PC_CODE': '01',
 'KGISDistri': '01',
 'AC_NAME_KN': 'ನಿಪ್ಪಾಣಿ',
 'Shape_STAr': 425991971.009,
 'Shape_STLe': 207237.810237}

In [62]:
# Fetch the constituency number and the corresponding constituency name from GeoJSON and convert to Dataframe

temp_dict = {}

for feature in acmap['features']:
    temp_dict.update({int(feature['properties']['AC_CODE']):feature['properties']['AC_NAME'].strip()})

In [63]:
# Fetch the constituency number and the corresponding constituency name from OpenCity dataset and convert to Dataframe

with open(f"{DIR}/2013 Elections\const_num.json",'r',encoding='utf-8') as file:
    const_num = json.loads(file.read())

In [66]:
# Correcting constituency names and numbers in the GeoJSON
for feature in acmap['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [67]:
# Saving the JSON
with open("AC_Boundary_simplified.json",'w',encoding='utf-8') as file:
    file.write(json.dumps(acmap))

In [68]:
with open("BBMP.json",'r',encoding='utf-8') as file:
    bbmp = json.loads(file.read())

In [69]:
# Correcting constituency names and numbers in the GeoJSON
for feature in bbmp['features']:
    feature['properties']['AC_NAME'] = const_num['Constituency Name'][feature['properties']['AC_CODE'].lstrip('0')]
    feature['properties']['AC_CODE'] = feature['properties']['AC_CODE'].lstrip('0')

In [70]:
# Saving the JSON
with open("BBMP.json",'w',encoding='utf-8') as file:
    json.dump(bbmp, file)

## Scaling the Banagalore region dues to it's high number of constituencies in a smaller area

In [71]:
# Define a scaling factor
scale_factor = 6.25

VERTICAL_OFFSET = 1.9
HORIZONTAL_OFFSET = 0.6

# Define center point of your map for scaling
# Update the code here to handle list of coordinates
num_coords = 0
total_lat = 0
total_lon = 0

for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            total_lat += coord[1]
            total_lon += coord[0]
            num_coords += 1

center_lat = total_lat / num_coords
center_lon = total_lon / num_coords

# Iterate over all coordinates and scale
for feature in bbmp['features']:
    for ring in feature['geometry']['coordinates']:
        for coord in ring:
            # Calculate offset from center point
            lat_offset = coord[1] - center_lat
            lon_offset = coord[0] - center_lon

            # Scale offset and recenter
            coord[1] = center_lat + (lat_offset * scale_factor) + VERTICAL_OFFSET
            coord[0] = center_lon + (lon_offset * scale_factor) + HORIZONTAL_OFFSET

# Save your scaled GeoJSON
with open('scaled_BBMP.json', 'w') as f:
    json.dump(bbmp, f)

In [72]:
with open('AC_Boundary_simplified.json', 'r') as f:
    acmap = json.load(f)

acmap['features'].extend(bbmp['features'])

# Save combined GeoJSON
with open('combined.json', 'w') as f:
    json.dump(acmap, f)

## Removing unnecessary properties from geoJSON to reduce size

In [ ]:
import json

# Function to remove unnecessary properties from features
def clean_geojson_properties(geojson_data, keep_properties=['AC_CODE', 'AC_NAME']):
    for feature in geojson_data['features']:
        if 'properties' in feature:
            # Keep only the required properties
            feature['properties'] = {k: v for k, v in feature['properties'].items() if k in keep_properties}
    return geojson_data

# Load the regular JSON file with GeoJSON structure
with open("combined.json", 'r', encoding = 'utf-8') as f:
    geojson_data = json.load(f)

# Clean the properties, keeping only AC_CODE and AC_NAME
geojson_data = clean_geojson_properties(geojson_data, keep_properties=['AC_CODE', 'AC_NAME'])

# Save the cleaned data back to a new JSON file
with open('output_cleaned.json', 'w', encoding = 'utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False)



## Reducing the precision of geo-coordinates to reduce size of the file

In [ ]:
import json

# Function to round coordinates to 6 decimal places
def round_coordinates(coordinates, decimal_places=6):
    if isinstance(coordinates[0], list):  # It's a list of lists (i.e., a polygon or multipolygon)
        return [round_coordinates(coord, decimal_places) for coord in coordinates]
    else:  # It's a simple coordinate (i.e., [longitude, latitude])
        return [round(coord, decimal_places) for coord in coordinates]

# Function to process the GeoJSON-like structure
def reduce_geojson_precision(geojson_data, decimal_places=6):
    for feature in geojson_data['features']:
        geometry = feature['geometry']
        if geometry['type'] == 'Point':
            geometry['coordinates'] = [round(coord, decimal_places) for coord in geometry['coordinates']]
        elif geometry['type'] == 'LineString' or geometry['type'] == 'MultiPoint':
            geometry['coordinates'] = [round_coordinates(coord, decimal_places) for coord in geometry['coordinates']]
        elif geometry['type'] == 'Polygon' or geometry['type'] == 'MultiLineString':
            geometry['coordinates'] = [round_coordinates(ring, decimal_places) for ring in geometry['coordinates']]
        elif geometry['type'] == 'MultiPolygon':
            geometry['coordinates'] = [[round_coordinates(ring, decimal_places) for ring in poly] for poly in geometry['coordinates']]
    
    return geojson_data

# Load the regular JSON file with GeoJSON structure
with open(r"output_cleaned.json", 'r', encoding = 'utf-8') as f:
    geojson_data = json.load(f)

# Reduce precision to 6 decimal places
geojson_data = reduce_geojson_precision(geojson_data, decimal_places=4)

# Save the modified data back to a new JSON file
with open('output.json', 'w', encoding = 'utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False)
